# Notebook Purpose
This notebook contains a handful of basic visualization about our datasette requests data. 

In [ ]:
import pandas as pd

import usage_metrics.schemas.datasette as datasette_schemas
from usage_metrics.resources.postgres import get_engine

%config InlineBackend.figure_format = 'retina'

In [ ]:
engine = get_engine()

with engine.connect() as con:
    raw_logs = pd.read_sql_table("raw_logs", con)

raw_logs.info()

In [ ]:
engine = get_engine()

with engine.connect() as con:
    unpack_httprequests = pd.read_sql_table("unpack_httprequests", con)

unpack_httprequests.info()

In [ ]:
fields = list(set(unpack_httprequests.columns) ^ set(raw_logs.columns))
fields

In [ ]:
engine = get_engine()

with engine.connect() as con:
    clean_datasette_logs = pd.read_sql_table("clean_datasette_logs", con)

clean_datasette_logs.info()

In [ ]:
fields = list(set(clean_datasette_logs.columns) ^ set(unpack_httprequests.columns))
fields

In [ ]:
clean_datasette_logs[fields].head()

In [ ]:
clean_datasette_logs.remote_ip_country_name.value_counts()

In [ ]:
clean_datasette_logs.remote_ip_region.value_counts()

In [ ]:
unpack_httprequests[fields].head()

In [ ]:
unpack_httprequests[fields].info()

In [ ]:
unpack_httprequests.columns[unpack_httprequests.isna().all()].to_list()

In [ ]:
unpack_httprequests.iloc[unpack_httprequests.response_size.idxmax()]

In [ ]:
datasette_schemas.raw_logs.validate(logs, lazy=True)

In [ ]:
from datetime import datetime

datetime(tzinfo="UTC")

In [ ]:
logs.isna().sum()

In [ ]:
logs.operation.iloc[200]

In [ ]:
logs[~logs.trace.isna()][["trace", "insert_id"]].iloc[0]

In [ ]:
from usage_metrics.schemas.datasette import raw_logs

## Request Volume
Number of requests for ferc1 and pudl tables and downloads per week.

In [ ]:
logs.set_index("timestamp").resample("1W").count().request_url_path.plot(
    xlabel="Week", ylabel="Number of Data Requests"
)

## Top Data Requests
The top datasette path requests. This includes json, csv and db downloads.

### Number of top data requests

In [ ]:
top_n = 20

logs.request_url_path.value_counts().head(top_n).sort_values().plot.barh(
    figsize=(20, 10),
    fontsize=20,
    ylabel="Number of Requests",
    title="Requests by Data Path",
)

### Percent of all requests

In [ ]:
top_n = 20

top_paths = logs.request_url_path.value_counts(normalize=True).head(top_n)

top_paths.sort_values().plot.barh(
    figsize=(20, 10),
    fontsize=20,
    ylabel="Percent of Requests",
    title="Requests by Data Path",
)

print(f"This chart shows {top_paths.sum() * 100} % of all data requests.")

## Top Organizations
Most organizations are generic internet providers however universities often have their own network set up. Unfortunately ipinfo charges for organization type so we have do some simple string filtering for now.  

In [ ]:
university_substrings = ["Universitaet", "University", "College", "Institute"]

is_uni = logs.remote_ip_org.str.contains("|".join(university_substrings))

logs[is_uni].remote_ip_org.value_counts().sort_values().plot.barh(
    figsize=(20, 10),
    fontsize=20,
    ylabel="Number of Requests",
    title="Requests by Academic Organization",
)

## Top Cities

In [ ]:
top_n = 20

full_location = (
    logs.remote_ip_city + ", " + logs.remote_ip_region + ", " + logs.remote_ip_country
)

full_location.value_counts().head(top_n).sort_values().plot.barh(
    figsize=(20, 10), fontsize=20, ylabel="Number of Requests", title="Requests by City"
)

In [ ]:
logs.dtypes